# @robotodio

---

## Libraries

In [81]:
# Python libraries
# ------------------------------------------------------------------------------
# Reading files with different formats
import json

# Data wrangling
import pandas as pd
import numpy as np

# Twitter API
import tweepy

# Hate speech detection
from detoxify import Detoxify

## Download data from twitter

The tweepy API class provides access to the entire Twitter RESTful API methods. Each method can accept various parameters and return responses.

### API set up

In [82]:
# API Twitter credentials
# ------------------------------------------------------------------------------
# Open .json file containing credentials/tokens as a dictionary
with open("twitter_api_keys.json") as file:
    api_credentials = json.load(file)
    
# Assign each value of the dictionary to a new variable
consumer_key = api_credentials['consumer_key']
consumer_secret = api_credentials['consumer_secret']
access_token = api_credentials['access_token']
access_token_secret = api_credentials['access_token_secret']

In [83]:
# API set up
# ------------------------------------------------------------------------------
# Create a handler instance with key and secret consumer, and pass the tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
    
# Construct the API instance
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Check credentials
if(api.verify_credentials):
    print("Logged In Successfully")
else:
    print("Error -- Could not log in with your credentials")

Logged In Successfully


### Extract tweets and associated metadata

Once the `tweepy.API` class has been instantiated we can call its methods to scrap desired information in Twitter.

Pagination is used a lot in Twitter API development. Iterating through timelines, user lists, direct messages, etc. In order to perform pagination, we must supply a page/cursor parameter with each of our requests. The problem here is this requires a lot of boiler plate code just to manage the pagination loop. To help make pagination easier and require less code, Tweepy has the Cursor object.

In first place, we are going to define a function that returns an iterator of tweets. We will call this fuction each time we want to download tweets from an user account.

In [106]:
# Tweets list iterator
# ------------------------------------------------------------------------------
def tweets_iterator(target, n_items):
    '''
    Returns an iterator of tweets.

        Parameters:
            target (str): The user name of the Twitter account.
            n_items (int): Number of tweets downloaded.

        Returns:
            tweets (ItemIterator): an iterator of tweets
    '''
    # Instantiate the iterator
    tweets = tweepy.Cursor(
        api.user_timeline,
        screen_name=target,
        include_rts=False,
        exclude_replies=False,
        tweet_mode='extended').items(n_items)
    
    # Returns iterator
    return tweets

In the next cell we can see an example of all the metadata that contains one tweet.

In [107]:
# Introduce the target Twitter account and number of items to download
target = 'lexfridman'
n_items = 20

# Tweets list (iterator)
tweets = tweets_iterator(target, n_items)

for k, tweet in enumerate(tweets):
    print(json.dumps(tweet._json, indent=4))
    if k == 1:
        break

{
    "created_at": "Tue Jan 12 01:32:04 +0000 2021",
    "id": 1348804908422754310,
    "id_str": "1348804908422754310",
    "full_text": "Accurate. https://t.co/UIbkQxyQoF",
    "truncated": false,
    "display_text_range": [
        0,
        9
    ],
    "entities": {
        "hashtags": [],
        "symbols": [],
        "user_mentions": [],
        "urls": [],
        "media": [
            {
                "id": 1348804642919100418,
                "id_str": "1348804642919100418",
                "indices": [
                    10,
                    33
                ],
                "media_url": "http://pbs.twimg.com/media/ErfqrSyXMAIaPKF.jpg",
                "media_url_https": "https://pbs.twimg.com/media/ErfqrSyXMAIaPKF.jpg",
                "url": "https://t.co/UIbkQxyQoF",
                "display_url": "pic.twitter.com/UIbkQxyQoF",
                "expanded_url": "https://twitter.com/lexfridman/status/1348804908422754310/photo/1",
                "type": "photo",


As we can see in the previous output cell, each tweet contains a lot of metadata. All this information comes in .json format, and we can access to it with Tweepy methods, or with dictionary methods.

In the next cell we are going to extract some metadata from each tweet, and export to a Pandas DataFrame format.

In [108]:
# Tweet extractor
# ------------------------------------------------------------------------------
# Tweets list (iterator)
tweets = tweets_iterator(target, n_items)

# Read through the iterator, and export the info to a Pandas DataFrame
all_columns = [np.array([
    tweet.full_text,
    tweet.user.screen_name,
    tweet.id,
    tweet.source,
    tweet.created_at,
    len(tweet.full_text),
    tweet.favorite_count,
    tweet.retweet_count,
    str(tweet.entities['hashtags'])
]) for tweet in tweets]

# Export the list of tweets to a dataframe
df = pd.DataFrame(
    data=all_columns,
    columns=['tweet', 'id', 'account', 'source', 'date', 'length', 'likes',
             'RTs', 'hashtags']
)

df.head()

,tweet,id,account,source,date,length,likes,RTs,hashtags
0,Accurate. https://t.co/UIbkQxyQoF,lexfridman,1348804908422754310,Twitter Web App,2021-01-12 01:32:04,33,10148,751,[]
1,Here's my 2nd conversation with Dmitry Korkin ...,lexfridman,1348585639630041092,Twitter Web App,2021-01-11 11:00:46,235,342,25,[]
2,When you turn your back on the voices of those...,lexfridman,1348306758452867075,Twitter Web App,2021-01-10 16:32:36,128,7296,792,[]
3,Life rarely happens exactly as planned. The fu...,lexfridman,1348297309436727296,Twitter Web App,2021-01-10 15:55:03,65,5545,540,[]
4,The math checks out. https://t.co/oepLfR8zok,lexfridman,1348110012904845321,Twitter Web App,2021-01-10 03:30:48,44,14355,1228,[]


**Incluir eliminado de duplicados y data cleaning en el texto.**

## Hate speech level prediction

`Detoxify` library has three pre-trained models:

- original: toxic, severe_toxic, obscene, threat, insult, identity_hate.
- unbiased: toxicity, severe_toxicity, obscene, threat, insult, identity_attack, sexual_explicit.
- multilingual: toxicity

While *original* and *unbiased* models generate different scorings like: toxicity, obscene, threat, insult...etc, *multilingual* model only produces a toxicity scoring. On the other hand, *multilingual* model is the only one that can bring a toxicity score in a language different from english.

Like we want to analyze any kind of accounts independently from the origin, and in order to make a more robust and scalable solution, we are going to chose *multilingual* one.

In [109]:
# Hate speech level prediction
# ------------------------------------------------------------------------------
# Returns a dictionary with toxicity values of each tweet. The key of the
# dictionary is called 'toxicity'.
results = Detoxify('multilingual').predict(list(df['tweet']))
results

{'toxicity': [0.0007464243681170046,
  0.0004143674741499126,
  0.22248923778533936,
  0.00045432784827426076,
  0.00042401894461363554,
  0.0004101789672859013,
  0.003279693890362978,
  0.001105726696550846,
  0.000854434329085052,
  0.02295275218784809,
  0.0004325560003053397,
  0.5668947696685791,
  0.0012916838750243187,
  0.0004201886767987162,
  0.0004094214818906039,
  0.00041339985909871757,
  0.0005079717375338078,
  0.00040876006823964417,
  0.0004219318798277527,
  0.00041506424895487726]}

In [110]:
# Export the new info to the previous DataFrame
df['toxicity'] = results['toxicity']
df.head()

,tweet,id,account,source,date,length,likes,RTs,hashtags,toxicity
0,Accurate. https://t.co/UIbkQxyQoF,lexfridman,1348804908422754310,Twitter Web App,2021-01-12 01:32:04,33,10148,751,[],0.000746
1,Here's my 2nd conversation with Dmitry Korkin ...,lexfridman,1348585639630041092,Twitter Web App,2021-01-11 11:00:46,235,342,25,[],0.000414
2,When you turn your back on the voices of those...,lexfridman,1348306758452867075,Twitter Web App,2021-01-10 16:32:36,128,7296,792,[],0.222489
3,Life rarely happens exactly as planned. The fu...,lexfridman,1348297309436727296,Twitter Web App,2021-01-10 15:55:03,65,5545,540,[],0.000454
4,The math checks out. https://t.co/oepLfR8zok,lexfridman,1348110012904845321,Twitter Web App,2021-01-10 03:30:48,44,14355,1228,[],0.000424


## X. Bibliografía

- http://docs.tweepy.org/en/latest/
- https://github.com/unitaryai/detoxify

## X. Info a revisar

- Limpieza de datos con regex: http://rios.tecnm.mx/cdistribuido/recursos/MinDatScr/MineriaScribble.html